In [1]:
import numpy as np
import pandas as pd

In [2]:
books = pd.read_csv('../data/raw/books.csv')
books.head()

,title,category,price,availability,rating,description,img
0,A Light in the Attic,Poetry,Â£51.77,True,3,It's hard to imagine a world without A Light i...,https://books.toscrape.com/media/cache/fe/72/f...
1,Tipping the Velvet,Historical Fiction,Â£53.74,True,1,"""Erotic and absorbing...Written with starling ...",https://books.toscrape.com/media/cache/08/e9/0...
2,Soumission,Fiction,Â£50.10,True,1,"Dans une France assez proche de la nÃ´tre, un ...",https://books.toscrape.com/media/cache/ee/cf/e...
3,Sharp Objects,Mystery,Â£47.82,True,4,"WICKED above her hipbone, GIRL across her hear...",https://books.toscrape.com/media/cache/c0/59/c...
4,Sapiens: A Brief History of Humankind,History,Â£54.23,True,5,From a renowned historian comes a groundbreaki...,https://books.toscrape.com/media/cache/ce/5f/c...


In [3]:
# lets find how many rows are there
books.shape

(1000, 7)

In [5]:
# missing  values 
books.isnull().sum()

title           0
category        0
price           0
availability    0
rating          0
description     0
img             0
dtype: int64

In [6]:
# duplicates 
books.duplicated().sum()

np.int64(0)

0 1 1 2 3 5 8 13 21 34 